In [1]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
tweets_main = pd.read_csv(f'../datasets/tweets_full.csv')
analyzer = SentimentIntensityAnalyzer()

In [3]:
#tweets = tweets_main.copy()

In [4]:
company = "AMZN"
tweets = tweets_main[tweets_main["ticker_symbol"] == company].copy().reset_index(drop=True)

In [5]:
def clean_body_v2(col):
    import re

    # usun urle
    # usun hashtagi
    # usun RT @username
    pattern_url = r"https?://[a-z0-9.]+/[?a-z0-9./-|]+"
    pattern_url_weak = r"https?://[a-z0-9.]+"
    pattern_www = r"www.[a-z0-9.]+/[?a-z0-9./-|]+"
    pattern_www_weak = r"www.[a-z0-9.]+"
    pattern_hash_dolla = r"[\$#][a-z0-9.]+"
    pattern_retweet = r"RT @[a-z0-9\S.]+"
    pattern_username = r"@[a-z0-9\S.]+"
    pattern_scraper_artifacts = r"[%-=_][\w+-\?\&|]+"

    col = col.str.replace(pattern_url, "", col, flags=re.I)
    col = col.str.replace(pattern_url_weak, "", col, flags=re.I)
    col = col.str.replace(pattern_www, "", col, flags=re.I)
    col = col.str.replace(pattern_www_weak, "", col, flags=re.I)
    col = col.str.replace(pattern_hash_dolla, "", col, flags=re.I)
    col = col.str.replace(pattern_retweet, "", col, flags=re.I)
    col = col.str.replace(pattern_username, "", col, flags=re.I)
    col = col.str.replace(pattern_scraper_artifacts, "", col, flags=re.I)
    col = col.str.replace("…", "", col, flags=re.I)


    return " ".join(col.split())

def clean_body(col):
    import re

    # usun urle
    # usun hashtagi
    # usun RT @username
    pattern_url = r"https?://[a-z.]+/[?a-z0-9./]+"
    pattern_hash_dolla = r"[\$#][a-z.]+"
    #pattern_retweet = r"RT @[a-z0-9\S.]+"
    #pattern_username = r"@[a-z0-9\S.]+"
    #pattern_scraper_artifacts = r"[%-=_][\w+-\?\&|]+"

    col = col.str.replace(pattern_url, "", col, flags=re.I)
    col = col.str.replace(pattern_hash_dolla, "", col, flags=re.I)
    #col = col.str.replace(pattern_retweet, "", col, flags=re.I)
    #col = col.str.replace(pattern_username, "", col, flags=re.I)
    #col = col.str.replace(pattern_scraper_artifacts, "", col, flags=re.I)
    #col = col.str.replace("…", "", col, flags=re.I)

    return col

def convert_dict_keyval_to_col(dict, key):
    new_col = []
    for i in dict:
        new_col.append(i[key])
    return new_col


def calc_weight(tweet):
    retweets = tweet.retweet_num
    likes = tweet.like_num
    comms = tweet.comment_num
    weight = 0
    if tweet.retweet_num == 0:
        weight = 1 + 0.05 * likes + 0.2 * comms
    else:
        weight = retweets * (1 + 0.05 * likes + 0.2 * comms)
    return weight


def calc_weight_2(tweet):
    retweets = tweet.retweet_num
    likes = tweet.like_num
    comms = tweet.comment_num

    weight = 1 + 0.5 * retweets + 0.05 * likes + 0.2 * comms
    return weight


def explore_weights(df, col):
    print(df[df[col] == max(df[col])].body)
    print(tweets[col].describe(percentiles=[0.05 * (i + 1) for i in range(19)]))


def percentage_weights(df, col):
    base = round(len(df.loc[df[col] == 1]) / len(df), 3) * 100
    one_three = len(df.loc[(df[col] > 1) & (df[col] < 3)]) / len(tweets) * 100
    three_five = len(df.loc[(df[col] > 3) & (df[col] < 10)]) / len(tweets) * 100

    print("Worth 1 tweet:", round(base, 3))
    print("Worth 1 to 3 tweets:", round(one_three, 3))
    print("Worth 3 to 5 tweets:", round(three_five, 3))
    print("Total: ", round(base, 3) + round(one_three, 3) + round(three_five, 3))


In [6]:
def clean_body_v3(col):
    import re

    # usun urle
    # usun hashtagi
    # usun RT @username
    pattern_url = r"https?://[a-z0-9.]+/[?a-z0-9./-|]+"
    pattern_url_weak = r"https?://[a-z0-9.]+"
    pattern_www = r"www.[a-z0-9.]+/[?a-z0-9./-|]+"
    pattern_www_weak = r"www.[a-z0-9.]+"
    pattern_hash_dolla = r"[\$#][a-z0-9.]+"
    pattern_retweet = r"RT @[a-z0-9\S.]+"
    pattern_username = r"@[a-z0-9\S.]+"
    pattern_scraper_artifacts = r"[%-=_][\w+-\?\&|]+"

    col = col.str.replace(re.compile(pattern_url, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_url_weak, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_www, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_www_weak, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_hash_dolla, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_retweet, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_username, flags=re.I), "", regex=True)
    col = col.str.replace(re.compile(pattern_scraper_artifacts, flags=re.I), "", regex=True)
    col = col.str.replace("…", "")

    return col.str.split().str.join(" ")

In [7]:
def calc_sentiment(df, to_preserve):
    df = df.copy()
    
    sent = df["body"].apply(analyzer.polarity_scores)

    sentiment_val = pd.DataFrame()
    sentiment_val["pos"] = convert_dict_keyval_to_col(sent, "pos")
    sentiment_val["neu"] = convert_dict_keyval_to_col(sent, "neu")
    sentiment_val["neg"] = convert_dict_keyval_to_col(sent, "neg")
    sentiment_val["comp"] = convert_dict_keyval_to_col(sent, "compound")

    for col in to_preserve:
        sentiment_val[col] = df[col]

    return sentiment_val

In [8]:
clean = tweets.copy()
clean.body = clean_body_v3(clean.body)


In [9]:
empty_mask = clean.body.str.len() == 0


In [10]:
clean[empty_mask].ticker_symbol.value_counts()

AMZN    6975
Name: ticker_symbol, dtype: int64

In [11]:
#SentimentIntensityAnalyzer().polarity_scores(tsla[tsla.tweet_id == 1048209850411438080].body)

In [12]:
#len(clean[xd == 0].body)/len(clean)

In [13]:
sentiment = calc_sentiment(
    tweets,
    to_preserve=[
        "post_date",
        "ticker_symbol",
        "comment_num",
        "retweet_num",
        "like_num",
    ],
)

In [14]:
sentiment["is_positive"] = 0 
sentiment.loc[sentiment["comp"] > 0.05, "is_positive"] = 1

sentiment["is_neutral"] = 0 
sentiment.loc[(sentiment["comp"] > -0.05) & (sentiment["comp"] < 0.05), "is_neutral"] = 1

sentiment["is_negative"] = 0 
sentiment.loc[sentiment["comp"] < -0.05, "is_negative"] = 1

In [44]:
sentiment["w1"] = sentiment.apply(calc_weight, axis=1)
sentiment["w2"] = sentiment.apply(calc_weight_2, axis=1)

In [47]:
#with open("lol.txt", "w") as f:
#    f.write(sentiment[:10].to_latex())
sentiment[:11]

,pos,neu,neg,comp,post_date,ticker_symbol,comment_num,retweet_num,like_num,is_positive,is_neutral,is_negative,w1,w2
0,0.000,0.876,0.124,-0.4278,2015-01-01 01:00:00,AMZN,0,0,0,0,0,1,1.00,1.00
1,0.096,0.904,0.000,0.2023,2015-01-01 01:00:00,AMZN,0,0,2,1,0,0,1.10,1.10
2,0.000,1.000,0.000,0.0000,2015-01-01 01:00:00,AMZN,0,0,0,0,1,0,1.00,1.00
3,0.000,0.632,0.368,-0.7506,2015-01-01 01:00:00,AMZN,21,139,57,0,0,1,1118.95,77.55
4,0.081,0.616,0.303,-0.6908,2015-01-01 01:00:00,AMZN,1,2,1,0,0,1,2.50,2.25
5,0.000,0.686,0.314,-0.7506,2015-01-01 01:00:00,AMZN,0,0,0,0,0,1,1.00,1.00
6,0.000,0.740,0.260,-0.7712,2015-01-01 01:00:00,AMZN,0,0,0,0,0,1,1.00,1.00
7,0.000,0.652,0.348,-0.7506,2015-01-01 01:00:00,AMZN,0,0,0,0,0,1,1.00,1.00
8,0.080,0.920,0.000,0.0772,2015-01-01 01:00:00,AMZN,0,0,0,1,0,0,1.00,1.00
9,0.000,0.632,0.368,-0.7506,2015-01-01 01:00:00,AMZN,0,0,0,0,0,1,1.00,1.00


In [48]:
tweets.iloc[196214].body

'Pattern Energy dedicates 150 MW #Amazon Wind Farm in Indiana. Read more: http://owler.us/aa70pg $AMZN'

In [49]:
clean.iloc[196214].body

'Pattern Energy dedicates MW Wind Farm in Indiana. Read more:'

In [50]:
# round down
def round_down_hour(col):
    try:
        date, time = col.split(" ")
        time = f"{time[0:2]}:00:00"
    except:
        print(col)
        raise Exception
    return date + " " + time


def aggregate_simple(df):
    aggregated = pd.pivot_table(
        df,
        index=["post_date"],
        aggfunc={
            "pos": ["max", "min", "std", "mean", "median", "count"],
            "neu": ["max", "min", "std", "mean", "median"],
            "neg": ["max", "min", "std", "mean", "median"],
            "comp": ["max", "min", "std", "mean", "median"],
            "comment_num": ["sum"],
            "retweet_num": ["sum"],
            "like_num": ["sum"],
            "is_positive": ["sum"],
            "is_negative": ["sum"],
            "is_neutral": ["sum"],
        },
    ).reset_index()

    aggregated.columns = [f"{i}_{j}" for i, j in aggregated.columns]
    # clean columns
    aggregated.rename(
        columns={
            f"post_date_": f"post_date",
            f"pos_count": f"count",
        },
        inplace=True,
    )

    return aggregated


def reweight(df, weight):
    cols = ["pos", "neu", "neg", "comp"]
    for col in cols:
        df[col] *= df[weight]
    return df


def get_total_tweets(df):
    df["post_date"] = df["post_date"].apply(round_down_hour)
    grouped = pd.pivot_table(
        df,
        index=["post_date"],
        aggfunc=["count"],
    ).reset_index()
    grouped.columns = [f"{i}_{j}" for i, j in grouped.columns]
    grouped.rename(
        columns={
            f"post_date_": f"post_date",
            f"count_body": f"total_count",
        },
        inplace=True,
    )
    return grouped[["post_date", "total_count"]]


def get_relative_count(df, df_total):

    mrg = df.merge(df_total, how="left", on="post_date")
    df["relative_count"] = mrg["count"] / mrg["total_count"]
    return df

def normalize_polarity(df):
    summed = df["is_negative_sum"] + df["is_neutral_sum"] + df["is_positive_sum"]
    df["is_negative_sum"] /= summed
    df["is_neutral_sum"] /= summed
    df["is_positive_sum"] /= summed
    df.fillna(0, inplace=True)
    return df

In [51]:
total_tweets = get_total_tweets(tweets_main)
total_tweets.head()

,post_date,total_count
0,2015-01-01 01:00:00,50
1,2015-01-01 02:00:00,38
2,2015-01-01 03:00:00,29
3,2015-01-01 04:00:00,30
4,2015-01-01 05:00:00,24


In [52]:
sentiment["post_date"] = sentiment["post_date"].apply(round_down_hour)

In [53]:
sentiment[sentiment["post_date"] == "2015-01-01 05:00:00"].head()

,pos,neu,neg,comp,post_date,ticker_symbol,comment_num,retweet_num,like_num,is_positive,is_neutral,is_negative,w1,w2
47,0.000,0.652,0.348,-0.7506,2015-01-01 05:00:00,AMZN,0,0,0,0,0,1,1.0,1.0
48,0.111,0.516,0.373,-0.7644,2015-01-01 05:00:00,AMZN,0,0,0,0,0,1,1.0,1.0
49,0.000,0.714,0.286,-0.7506,2015-01-01 05:00:00,AMZN,0,0,0,0,0,1,1.0,1.0
50,0.000,0.714,0.286,-0.7506,2015-01-01 05:00:00,AMZN,0,0,0,0,0,1,1.0,1.0
51,0.000,0.632,0.368,-0.7506,2015-01-01 05:00:00,AMZN,0,0,0,0,0,1,1.0,1.0


In [54]:
sentiment.sample(10)

,pos,neu,neg,comp,post_date,ticker_symbol,comment_num,retweet_num,like_num,is_positive,is_neutral,is_negative,w1,w2
369791,0.231,0.769,0.0,0.6249,2017-09-15 19:00:00,AMZN,0,2,1,1,0,0,2.10,2.05
532170,0.177,0.823,0.0,0.7845,2018-10-02 22:00:00,AMZN,0,1,14,1,0,0,1.70,2.20
168135,0.145,0.855,0.0,0.1779,2016-05-17 19:00:00,AMZN,0,0,0,1,0,0,1.00,1.00
433792,0.255,0.745,0.0,0.8020,2018-02-21 18:00:00,AMZN,0,0,1,1,0,0,1.05,1.05
423939,0.000,1.000,0.0,0.0000,2018-01-31 10:00:00,AMZN,0,0,0,0,1,0,1.00,1.00
620738,0.082,0.918,0.0,0.1779,2019-04-08 19:00:00,AMZN,0,1,0,1,0,0,1.00,1.50
104335,0.253,0.747,0.0,0.5267,2015-12-01 20:00:00,AMZN,0,0,0,1,0,0,1.00,1.00
435452,0.243,0.757,0.0,0.5423,2018-02-25 04:00:00,AMZN,1,0,3,1,0,0,1.35,1.35
481495,0.219,0.781,0.0,0.4939,2018-06-05 12:00:00,AMZN,0,0,0,1,0,0,1.00,1.00
662900,0.332,0.668,0.0,0.8159,2019-07-28 18:00:00,AMZN,0,0,0,1,0,0,1.00,1.00


In [55]:
agg_no_w = aggregate_simple(sentiment)
agg_no_w = get_relative_count(agg_no_w, total_tweets)
# std is NaN when count is 1
agg_no_w = agg_no_w.fillna(0)

In [56]:
agg_no_w.iloc[0]

post_date          2015-01-01 01:00:00
comment_num_sum                     22
comp_max                        0.2023
comp_mean                    -0.487573
comp_median                    -0.7506
comp_min                       -0.7712
comp_std                      0.387796
is_negative_sum                      8
is_neutral_sum                       1
is_positive_sum                      2
like_num_sum                        60
neg_max                          0.368
neg_mean                         0.223
neg_median                       0.303
neg_min                            0.0
neg_std                       0.159292
neu_max                            1.0
neu_mean                      0.753636
neu_median                       0.686
neu_min                          0.616
neu_std                        0.14296
count                               11
pos_max                          0.096
pos_mean                      0.023364
pos_median                         0.0
pos_min                  

In [57]:

with open("lol.txt", "w") as f:
    f.write(agg_no_w.iloc[0].to_latex())

C:\Users\Patryk\AppData\Local\Temp\ipykernel_17812\3468160594.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(agg_no_w.iloc[0].to_latex())


In [58]:
len(agg_no_w.columns)

29

In [59]:
agg_no_w = normalize_polarity(agg_no_w)
agg_no_w.iloc[[5920, 6251, 12361, 27423, 40990]]

,post_date,comment_num_sum,comp_max,comp_mean,comp_median,comp_min,comp_std,is_negative_sum,is_neutral_sum,is_positive_sum,...,neu_min,neu_std,count,pos_max,pos_mean,pos_median,pos_min,pos_std,retweet_num_sum,relative_count
5920,2015-09-17 19:00:00,2,0.7550,0.037272,0.0000,-0.6705,0.257415,0.186047,0.534884,0.279070,...,0.553,0.147626,43,0.412,0.066930,0.000,0.000,0.104035,2,0.281046
6251,2015-10-01 22:00:00,1,0.5574,-0.066374,0.0000,-0.6114,0.300981,0.315789,0.473684,0.210526,...,0.602,0.129968,19,0.254,0.052211,0.000,0.000,0.073383,4,0.155738
12361,2016-06-22 08:00:00,0,0.1779,0.177900,0.1779,0.1779,0.000000,0.000000,0.000000,1.000000,...,0.914,0.000000,1,0.086,0.086000,0.086,0.086,0.000000,0,0.012195
27423,2018-04-12 23:00:00,2,0.8481,0.188294,0.0772,-0.5423,0.418878,0.176471,0.294118,0.529412,...,0.533,0.150576,17,0.334,0.101588,0.091,0.000,0.120290,9,0.207317
40990,2019-12-29 01:00:00,13,0.9062,0.259320,0.3338,-0.4278,0.468089,0.200000,0.200000,0.600000,...,0.690,0.105044,10,0.310,0.099700,0.091,0.000,0.098876,6,0.196078


In [60]:
#agg_w1.to_csv(f"../datasets/v3/more_cols/w1/{company}.csv", index=False)
#agg_no_w.to_csv(f"../datasets/v3/senti/{company}.csv", index=False)